# Documentation for Standard Keras Neural Network
- ROC AUC for different masses
- learning curve
- validation curves

# Standard imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import HpAlgorithms
import HpMLUtils
import HpMLPlots
import HpHyperParameterOptimisation
import HpTrainingFrame

In [2]:
df_mc=pd.read_hdf('/eos/user/j/jglatzer/L2Output_R20_Nov_v2/pandas_ge6jge4b.h5', 'INC_ge6jge4b')

In [3]:
df_mc.columns

Index([u'index', u'Muu_MindR_70', u'nJets', u'Mbb_MindR_70', u'HT_jets',
       u'H1_all', u'Mjjj_MaxPt', u'Mbb_MaxPt_70', u'dRbb_avg_70',
       u'eventNumber', u'pT_jet5', u'dRlepbb_MindR_70', u'nBTags_70',
       u'Mbb_MaxM_70', u'jet_pt', u'Centrality_all', u'weight', u'process',
       u'group', u'region', u'pT_jet1', u'hpmass'],
      dtype='object')

# Prepare data for H+ classification

split into test and training data

In [7]:
from keras.models import Sequential
from keras.layers import Dense

def create_keras_model():
    # create model
    model = Sequential()
    model.add(Dense(14, input_dim=14, activation='relu'))
    model.add(Dense(5, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

from keras.wrappers.scikit_learn import KerasClassifier
model = KerasClassifier(build_fn=create_keras_model, epochs=150, batch_size=10, verbose=0)

# split into input (X) and output (Y) variables
#X = dataset[:,0:8]
#Y = dataset[:,8]
# create model

# evaluate using 10-fold cross validation
#kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
#results = cross_val_score(model, X, Y, cv=kfold)
#print(results.mean())
#clf=HpAlgorithms.getGradientBDTClassifier()#options={"n_estimators":10})

In [6]:
htf=HpTrainingFrame.HpTrainingFrame(df_mc) 

In [ ]:
X_train, X_test, X_eval, y_train, y_test,y_eval, w_train, w_test, w_eval=htf.prepare(hpmass=800)
model.fit(X_train,y_train)